In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
import pandas as pd
from konlpy.tag import Hannanum
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [2]:
hannanum = Hannanum()

In [3]:
df_stat = pd.read_excel('dongduk_course_manage.xlsx')
df_stat['학수번호'] = df_stat['학수번호'].str.strip() #학수번호 공백없애기 
df_stat = df_stat[['학수번호','과목소개']] 
df_stat.head()

,학수번호,과목소개
0,경영K0030,"고용관계론 사무직, 관리직, 비정규직, 첨단인력과 비노조 경영을 포괄하는 ..."
1,경영K0017,경영정보시스템 조직운영 및 경영활동을 지원하는 경영정보시스템에 관련된 학문...
2,경영K0036,"조직이론 조직행동기초이론을 배우고, 이를 활용하는 방법을 연구한다. 경..."
3,경영K0012,"소비자행동분석 소비자 행동의 다양한 측면을 이해토록 함으로써, 소비자를 대..."
4,경영K0025,"생산/물류관리 품질관리, 재고관리, 물류 분야에서 기업의 생산성을 개선하는..."


In [4]:
df_stat.tail()

,학수번호,과목소개
35,경영K0031,파생상품론 파생상품중 대표적인 상품인 옵션과 선물에 중점을 두어 학습하며 ...
36,경영K0035,전략경영모델 한국 기업에 대한 종합적인 이해력을 증진할 뿐 아니라 창의적인...
37,공예K0010,금속정밀주조 금속캐스팅은 수공예뿐 아니라 산업으로 연계한 금속공예의 역할...
38,공예K0018,기초금속공예II 금속공예의 연구심화과정으로 금속과 금속의 어울림을 통해 미...
39,공예K0032,산업도자II 도자의 재료인 석고에 대한 이해와 석고틀 제작기법을 연구하여 ...


In [5]:
docs = []
for i in df_stat['과목소개']:  
    docs.append(hannanum.nouns(i))
    
for i in range(len(docs)):
    docs[i] = ' '.join(docs[i])
df_stat['과목소개'] = pd.Series(docs)
df_stat.head()#token화

,학수번호,과목소개
0,경영K0030,고용관계론 사무직 관리직 정규직 첨단인력 노조 경영 포괄 종업원 경영자 사이 근 조...
1,경영K0017,경영정보시스템 조직운영 경영활동 지원 경영정보시스템 관련 학문적 이론 정보시스템 현...
2,경영K0036,조직이론 조직행동기초이론 이 활용 방법 연구 경영학 기초 것 통계학 중심 수리적 접...
3,경영K0012,소비자행동분석 소비자 행동 다양한 측면 이해토록 소비자 대상 마케팅 활동 시스템적 ...
4,경영K0025,생산 물류관리 품질관리 재고관리 물류 분야 기업 생산성 개선 방안 연구 이 응용 방...


In [7]:
df_ncs = pd.read_excel('ncs_edit2.xlsx',encoding='cp949')
df_ncs.head(3)

,jobname,capability
0,경영기획,경쟁사 대비 자사의 핵심역량 분석을 위해 기업의 제품 강점과 약점을 파악할 수 있다...
1,경영평가,"경영평가 관련 정보수집이란, 정보수집의 방법을 정하여 평가대상에 대한 정보와, 유사..."
2,PR,온라인 매체별 구성요소를 확정하여 제작·개선 계획을 수립할 수 있다.온라인 콘텐츠 ...


In [8]:
df_ncs.columns = ['jobname','capability']

In [9]:
df_ncs[:11]

,jobname,capability
0,경영기획,경쟁사 대비 자사의 핵심역량 분석을 위해 기업의 제품 강점과 약점을 파악할 수 있다...
1,경영평가,"경영평가 관련 정보수집이란, 정보수집의 방법을 정하여 평가대상에 대한 정보와, 유사..."
2,PR,온라인 매체별 구성요소를 확정하여 제작·개선 계획을 수립할 수 있다.온라인 콘텐츠 ...
3,광고,광고 전략 수립에 필요한 정치‧경제‧사회‧기술적 환경을 분석할 수 있다.광고 전략 ...
4,마케팅전략기획,신상품개발 필요성에 따라 신규 아이템 아이디어를 수집할 수 있다.수집된 정보와 개발...
5,회계_회계감사,"전표관리란 회계상 거래를 인식하고, 전표작성 및 이에 따른 증빙서류를 처리 및 관리..."
6,마케팅고객관리,"고객관리 계획수립은 효과적으로 고객을 관리하기 위하여 고객관리 전략목표를 도출하고,..."
7,귀금속가공,귀금속 제품생산기획이란 귀금속 주문제품을 생산하기 위하여 디자인 및 재료를 파악하고...
8,마케팅통계조사,통계조사계획이란 조사목적에 부합하는 정확한 조사를 수행하기 위하여 고객 요구사항 분...
9,총무,조직의 비전과 중·장기 사업전략에 따라 대ㆍ내외 산업 환경을 분석할 수 있다회사의 ...


In [10]:
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [11]:
courses_contents = [] 
def append_courses(df_name): #df이름을 매개변수로 
    courses = input('들은 과목들의 학수번호 (쉼표 기준):').split(',')
    return     df_name[df_name['학수번호'].isin(courses)]['과목소개'].sum()


In [12]:
#Countvect
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
    
courses_contents = [] 
def append_courses(df_name): #과목df이름을 매개변수로 
    courses = input('들은 과목들의 학수번호 (쉼표 기준):').split(',')
    return     df_name[df_name['학수번호'].isin(courses)]['과목소개'].sum()

    
def cv_recommend_ten_job():
    cv = CountVectorizer()
    distance = []
    courses = append_courses(df_stat)
    for k in range(0,df_ncs.shape[0]):
        X = np.array(cv.fit_transform([courses, df_ncs.iloc[k,1]]).todense())
        distance.append(cos_sim(X[0], X[1]))
    df = pd.DataFrame(distance, index=df_ncs['jobname'],columns=['recommended job'])
    df = df.sort_values(by=['recommended job'], axis=0,ascending=False)
    return df[:10]

In [13]:
#TF-IDF
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

courses_contents = [] 
def append_course(): 
    courses = input('들은 과목들의 학수번호 (쉼표 기준):').split(',')
    for r in courses:
        index = df_stat.index[df_stat['학수번호'] == r ].tolist()  #학수번호 인덱스뽑기
        #print(index)
        k = df_stat[df_stat['학수번호'] == r ]['과목소개'][index[0]]  #과목소개 추출
        #print(k)
        courses_contents.append(k)  #course_contents리스트에 에펜드
        #print(courses_contents)
    return ''.join(courses_contents)

    
def cv_recommend_ten_job():
    cv = TfidfVectorizer()
    distance = []
    courses = append_courses(df_stat)
    #print(courses)
    for k in range(0,df_ncs.shape[0]):
        X = np.array(cv.fit_transform([courses, df_ncs.iloc[k,1]]).todense())
        distance.append(cos_sim(X[0], X[1]))
    df = pd.DataFrame(distance, index=df_ncs['jobname'],columns=['recommended job'])
    df = df.sort_values(by=['recommended job'], axis=0,ascending=False)
    
    return df[:10]

In [15]:
cv_recommend_ten_job()

들은 과목들의 학수번호 (쉼표 기준):공예K0010,공예K0018,공예K0032


,recommended job
jobname,
귀금속가공,0.008360
PR,0.004288
광고,0.003987
마케팅통계조사,0.003749
경영평가,0.002746
품질관리,0.001789
총무,0.001744
물류관리,0.001581
인사,0.001507


In [ ]:
# 데이터 탐색 -> 워드클라우드 
# 유사도 시각화 -> heatmap
# 문제정의단계 -> 구글폼 학생들